Việc phân loại tập dữ liệu và áp dụng một chức năng cho mỗi nhóm, cho dù là tổng hợp hay chuyển đổi, thường là một thành phần quan trọng của quy trình phân tích dữ liệu.Sau khi tải, hợp nhất và chuẩn bị tập dữ liệu, bạn có thể cần tính toán thống kê nhóm hoặc có thể là bảng tổng hợp cho mục đích báo cáo hoặc trực quan hóa. Pandas cung cấp 1 flexible `group by` interface

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### 10.1 GroupBy Mechanics

Hadley Wickham, tác giả của nhiều gói phổ biến cho ngôn ngữ lập trình R, đã đặt ra thuật ngữ `split-apply-merge` để mô tả các hoạt động của `group()`. Trong giai đoạn đầu tiên của quy trình, dữ liệu chứa trong đối tượng **pandas**, dù là Series, DataFrame hay cách khác, được `split()` các nhóm dựa trên một hoặc nhiều khóa mà bạn cung cấp. Việc phân tách được thực hiện trên một trục cụ thể của một đối tượng. Ví dụ: một DataFrame có thể được nhóm trên các hàng (trục = 0) hoặc các cột của nó (trục = 1). Khi điều này được thực hiện, một hàm được áp dụng cho mỗi nhóm, tạo ra một giá trị mới. Cuối cùng, kết quả của tất cả các ứng dụng chức năng đó được `combine()` thành một đối tượng kết quả. Hình thức của đối tượng kết quả thường sẽ phụ thuộc vào những gì đang được thực hiện với dữ liệu ”

In [1]:
from IPython.display import Image
Image(url="./Images/Illustration of a group aggregation.png")

Mỗi nhóm key có thể lấy rất nhiều forms, và các key không có tất cả cùng kiểu loại

1. 1 List hoặc 1 mảng của giá trị rằng có thể cùng 1 chiều với trục đang nhóm

2. Một giá trị cho biết tên cột trong 1 Data frame

3. A dict hoặc 1 series cho bạn Đưa ra sự tương ứng giữa các giá trị trên trục được nhóm và tên nhóm"

4. A hàm có thể thực thi trên 1 axis indexx hoặc label trong index

*Lưu ý rằng ba phương pháp sau là các phím tắt để tạo ra một mảng giá trị được sử dụng để chia nhỏ đối tượng*

**Example**

In [3]:
df = pd.DataFrame({
    'key1': ['a','a','b','b','a'],
    'key2': ['one','two','one','two','one'],
    'data1': np.random.randn(5),
    'dat2': np.random.randn(5)
})

In [4]:
df

,key1,key2,data1,dat2
0,a,one,0.322583,0.494414
1,a,two,-1.739404,-0.498583
2,b,one,0.387045,0.485014
3,b,two,-0.409632,-0.317464
4,a,one,-0.428946,-1.110477


Giả sử bạn muốn tính giá trị trung bình của cột **data1** bằng cách sử dụng các nhãn từ **key1**. Có một số cách để làm điều này. Một là truy cập data1 và gọi nhóm theo cột (a Series) tại key1:

In [5]:
grouped = df['data1'].groupby(df['key1'])

In [7]:
grouped


biến `grouped ` là một **GroupBy Object**. Nó chưa thực sự tính toán bất cứ điều gì cả, nó chỉ gộp các nhóm lại với nhau thôi 

In [8]:
grouped.mean()

key1
a   -0.615256
b   -0.011293
Name: data1, dtype: float64

Giờ chúng ta sẽ giải thích cái gì đã xảy ra `.mean()`. Điều quan trọng ở đây là a `Series` đã gộp lại theo group `key`, sẽ làm ra 1 new `Series mới` thực hiện gộp các index trong `key1`. Kết quả sẽ là index của `key1` đã có



Nếu thay vì chúng ta truyền nhiều array như 1 list, chúng ta sẽ truyền nhiều tham số như 1 list chúng ta sẽ đc 1 cái khác biệt

In [12]:
df

,key1,key2,data1,dat2
0,a,one,0.322583,0.494414
1,a,two,-1.739404,-0.498583
2,b,one,0.387045,0.485014
3,b,two,-0.409632,-0.317464
4,a,one,-0.428946,-1.110477


In [14]:

means = df['data1'].groupby([df['key1'],df['key2']]).mean()


In [15]:
means

key1  key2
a     one    -0.053182
      two    -1.739404
b     one     0.387045
      two    -0.409632
Name: data1, dtype: float64

now chúng ta đang gộp dữ liệu với two keys. 

In [16]:
means.unstack()

key2,one,two
key1,,
a,-0.053182,-1.739404
b,0.387045,-0.409632


Trong ví dụ này, các khóa nhóm đều là Chuỗi, mặc dù chúng có thể là bất kỳ mảng nào có độ dài phù hợp:”

In [17]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])

In [18]:
years = np.array([2005,2005,2006,2005,2006])

In [19]:
df['data1'].groupby([states,years]).mean()

California  2005   -1.739404
            2006    0.387045
Ohio        2005   -0.043525
            2006   -0.428946
Name: data1, dtype: float64

In [20]:
df

,key1,key2,data1,dat2
0,a,one,0.322583,0.494414
1,a,two,-1.739404,-0.498583
2,b,one,0.387045,0.485014
3,b,two,-0.409632,-0.317464
4,a,one,-0.428946,-1.110477


Thông tin nhóm thường được tìm thấy trong DataFrame giống với dữ liệu bạn muốn làm việc. Trong trường hợp đó, bạn có thể chuyển tên cột (cho dù đó là chuỗi, số hay các đối tượng Python khác) làm khóa nhóm

In [21]:
df.groupby('key1').mean()

,data1,dat2
key1,,
a,-0.615256,-0.371549
b,-0.011293,0.083775


In [23]:
df

,key1,key2,data1,dat2
0,a,one,0.322583,0.494414
1,a,two,-1.739404,-0.498583
2,b,one,0.387045,0.485014
3,b,two,-0.409632,-0.317464
4,a,one,-0.428946,-1.110477


In [22]:
df.groupby(['key1','key2']).mean()

data1      dat2
key1 key2                    
a    one  -0.053182 -0.308032
     two  -1.739404 -0.498583
b    one   0.387045  0.485014
     two  -0.409632 -0.317464

Group chỉ áp dụng với dữ liệu số 

`size()` để trả về kích thước tổng pt nó gộp vào 

In [25]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [28]:
df.groupby('key1').size()

key1
a    3
b    2
dtype: int64

### Iterating Over Groups (Lặp các nhóm)
Đối tượng GroupBy hỗ trợ lặp lại, tạo ra một chuỗi `2 tuple` chứa `tên``nhóm` cùng với phần dữ liệu. Hãy xem xét những điều sau: ”

In [26]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1      dat2
0    a  one  0.322583  0.494414
1    a  two -1.739404 -0.498583
4    a  one -0.428946 -1.110477
b
  key1 key2     data1      dat2
2    b  one  0.387045  0.485014
3    b  two -0.409632 -0.317464


Trong trường hợp của nhiều key, phần tử đầu tiên trong `tuple` sẽ là 1 tuple của key xem ví dụ ở dứoi 

In [29]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2)) # Key of group 
    print(group) # data

('a', 'one')
  key1 key2     data1      dat2
0    a  one  0.322583  0.494414
4    a  one -0.428946 -1.110477
('a', 'two')
  key1 key2     data1      dat2
1    a  two -1.739404 -0.498583
('b', 'one')
  key1 key2     data1      dat2
2    b  one  0.387045  0.485014
('b', 'two')
  key1 key2     data1      dat2
3    b  two -0.409632 -0.317464


## Selecting a Column or Subset of Columns Chọn một cột hoặc một tập hợp con các cột

indexing a Groupby Object đã tạo từ 1 DataFrame với 1 tên một cột của mảng của cột tên có hiệu ứng subsetting cho 1 kết hợp: 
Nghĩa là mình chỉ group 1 cột thôI

In [30]:
df.groupby('key1')['data1']

In [32]:
df.groupby('key1')[['dat2']] 

In [33]:
df

,key1,key2,data1,dat2
0,a,one,0.322583,0.494414
1,a,two,-1.739404,-0.498583
2,b,one,0.387045,0.485014
3,b,two,-0.409632,-0.317464
4,a,one,-0.428946,-1.110477


In [36]:
# Gộp 2 key và chỉ tính mean ở dat2 
df.groupby(['key1','key2'])[['dat2']].mean()

dat2
key1 key2          
a    one  -0.308032
     two  -0.498583
b    one   0.485014
     two  -0.317464

Đối tượng được trả về bởi thao tác lập chỉ mục này là DataFrame được nhóm lại nếu một danh sách hoặc mảng được chuyển hoặc một Chuỗi được nhóm nếu chỉ một tên cột được chuyển dưới dạng một đại lượng vô hướng

In [37]:
s_grouped = df.groupby(['key1','key2'])['dat2']

In [38]:
s_grouped

In [39]:
s_grouped.mean()

key1  key2
a     one    -0.308032
      two    -0.498583
b     one     0.485014
      two    -0.317464
Name: dat2, dtype: float64

### Group with Dicts and Series
Nhóm thông tin có thể tồn tại ở dạng khác với mảng.

In [40]:
people = pd.DataFrame(np.random.randn(5,5),
                         columns =['a','b','c','d','e'],
                         index = ['Joe','Steve','Wes','Jim','Travis'])

In [41]:
people

,a,b,c,d,e
Joe,-0.137403,0.115970,1.554876,-0.013031,-0.570080
Steve,0.448430,0.327188,0.217527,1.812497,0.364040
Wes,-1.029458,-0.048496,-1.461664,-0.308536,0.629691
Jim,-0.913505,0.021841,0.229922,-0.091299,0.235635
Travis,0.551686,0.763015,-1.449026,0.672701,0.013421


In [42]:
people.iloc[2:3,[1,2]]= np.nan

In [43]:
people

,a,b,c,d,e
Joe,-0.137403,0.115970,1.554876,-0.013031,-0.570080
Steve,0.448430,0.327188,0.217527,1.812497,0.364040
Wes,-1.029458,NaN,NaN,-0.308536,0.629691
Jim,-0.913505,0.021841,0.229922,-0.091299,0.235635
Travis,0.551686,0.763015,-1.449026,0.672701,0.013421


Giả sử tôi có một nhóm tương ứng cho các cột và muốn tổng hợp các cột lại với nhau theo nhóm"

In [44]:
mapping = {
    'a':'red',
    'b':'red',
    'c':'blue',
    'd':'blue',
    'e':'red',
    'f':'orange'
}

bây giờ, bạn có thể xây dựng một mảng từ dit để truyền cho groupby 

In [48]:
by_column = people.groupby(mapping,axis=1)


In [49]:
by_column

In [50]:
by_column.sum()

,blue,red
Joe,1.541845,-0.591513
Steve,2.030024,1.139658
Wes,-0.308536,-0.399768
Jim,0.138624,-0.656029
Travis,-0.776325,1.328121


In [51]:
by_column = people.groupby(mapping)

In [52]:
by_column.sum()

,a,b,c,d,e


In [53]:
map_series = pd.Series(mapping)

In [54]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [55]:
people.groupby(map_series,axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


In [56]:
people

,a,b,c,d,e
Joe,-0.137403,0.115970,1.554876,-0.013031,-0.570080
Steve,0.448430,0.327188,0.217527,1.812497,0.364040
Wes,-1.029458,NaN,NaN,-0.308536,0.629691
Jim,-0.913505,0.021841,0.229922,-0.091299,0.235635
Travis,0.551686,0.763015,-1.449026,0.672701,0.013421


### Grouping with Functions 

Sử dụng các hàm Python là một cách chung chung hơn để xác định 
ánh xạ nhóm so với một dict hoặc Series. Bất kỳ hàm nào được truyền dưới dạng khóa nhóm sẽ được gọi một lần cho mỗi giá trị chỉ mục, với các giá trị trả về được sử dụng làm tên nhóm. Cụ thể hơn, hãy xem xét ví dụ DataFrame từ phần trước, có tên của mọi người làm giá trị chỉ mục. Giả sử bạn muốn nhóm theo độ dài của tên; trong khi bạn có thể tính một mảng độ dài chuỗi, đơn giản hơn là chỉ cần chuyển hàm len: ”

In [58]:
people.groupby(len).sum()

,a,b,c,d,e
3,-2.080367,0.137811,1.784798,-0.412866,0.295246
5,0.448430,0.327188,0.217527,1.812497,0.364040
6,0.551686,0.763015,-1.449026,0.672701,0.013421


### Grouping by Index Levels 

    Một tiện ích cuối cùng cho các tập dữ liệu được lập chỉ mục phân cấp là khả năng tổng hợp bằng cách sử dụng một trong các cấp của chỉ mục trục. Hãy xem một ví dụ: ”


In [59]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                        [1, 3, 5, 1, 3]],
                                        names=['cty', 'tenor'])



In [60]:
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [61]:
hier_df = pd.DataFrame(np.random.randn(4,5),
                      columns=columns)


In [62]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.160358  1.295980  1.621729 -1.139367  0.157585
1     -0.256707 -0.997242 -1.406137  0.877465 -0.456513
2      0.491722 -0.458101  0.979177 -0.615286  1.182135
3      0.231708  0.428897  0.278852  1.319950  1.058953

In [63]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3
